# **Trying to find content of an article from PubMed using Key words**

The code consists of the following Task

Identifying what is the content of article witout a scientific name of corresponding synonymn,common or pharmaceutical name.
This code analyses missing scientific name in an article is legit or if the artcle content is not relevant to the same by identifying the keywords

Required Installations

In [ ]:
!pip install biopython
!pip install beautifulsoup4
!pip3 install pytextrank
!spacy download en_core_web_sm
!pip3 install wordcloud
!pip install matplotlib==3.1.3
!pip3 install keybert


import dependent libraries

In [ ]:
import pandas as pd
import re
import Bio
import requests
from bs4 import BeautifulSoup
from Bio import Entrez
Entrez.email =  "radhu.palliyana@gmail.com" # provide mail id after creating api key

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

**Extracting article id and content of the articles related to medicinal plants for given species and genus with retrival max of 50 Ids**

Contains code for searching a term using Esearch.

It search for articles that talk about medicinal plants using the Esearch by giving term parameter and retreive article id.Then fectches the article content using Efetch using the corresponding article id .The text of article content is then extracted from abstract and body of article.

In [ ]:
handle = Entrez.esearch(db ="pmc", term= 'abelmoschi corolla',retmax= "50") # retrieve maximum 50 article ids containing the given term
rec_list = Entrez.read(handle)
handle.close()
print(rec_list['Count']) # retrieves the total number of article having the given term
print(len(rec_list['IdList']))
print(rec_list['IdList'])
total_id = rec_list['IdList'] #consolidating all the corresponding article Ids having the given term into a list
#extracting the article having the given term using each article id from the list

For eg: '9340222', '9403506', '8069016', '8037085', '7535141', '7482509', '6501976', '6500631', '6425497', '6222764', '6155732', '4926151' are the list of article corresponding to the given pharmaceutical name: abelmoschi corolla

Among these the following article have scientific names tagged to this pharmaceutical name:  9403506,7535141 and the rest doesnt have one.

So here for further research we take the one of the article which do not have scientific name mentioned and further investigate on the same. Article id : 8037085 is considered for the same

To get check for each article manually navigate to and replace the PMC iD with the article id number for eg for id 9340222 :https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9340222/

In [ ]:

handle = Entrez.efetch(db='pmc', id = 8037085 , retmode = 'xml') #Retrieve the corresponding article using article id
total_content =  handle.read()
#print(total_content) # Prints the entire html page content of the article.This is usefull to identify the corresponding tag containing the abstract and body of the article.
soup = BeautifulSoup(total_content,"html.parser")
abstracts = soup.find('abstract')
body = soup.find('body')
print(abstracts)
#extracting the text from abstract and body of the article
print("abstract for the article id :", id)
print(str(abstracts.string))
abstract_text = str(abstracts.string)
print("Entire content of the article id :", id)
print(body.get_text())
body_text = body.get_text()


Text Rank : Using text rank extracts and summerize the text of the article body

In [ ]:
import spacy
import pytextrank
# load a spaCy model
nlp = spacy.load("en_core_web_sm")
# add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")
abstract_doc = nlp(body_text)
# examine the top-ranked phrases in the document
for phrase in abstract_doc._.phrases[:30]:
    print(phrase.text)

Word Cloud

In [ ]:
import collections
import numpy as np
import pandas as pd
#import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib import rcParams
from wordcloud import WordCloud, STOPWORDS

stopwords = STOPWORDS
wordcloud = WordCloud(stopwords=stopwords, background_color="white", max_words=1000).generate(body_text)
rcParams['figure.figsize'] = 15, 20
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

Keybert is used for keyword extraction from article. It generates most similar keywords and keyphrases to a given document using BERT embedding

In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(body_text)
print("keywords body",keywords)
keywords_abstract = kw_model.extract_keywords(abstract_text)
print("keywords abstract",keywords_abstract)

**Reference:**
Entrez is a molecular biology database system that provides integrated access to nucleotide and protein sequence.The system is produced by the National Center for Biotechnology Information (NCBI).

Entrez Programming Utilities user guide is available at : https://www.ncbi.nlm.nih.gov/books/NBK25501/

PyTextRank documentation: https://pypi.org/project/pytextrank/

WordCloud documentation: https://amueller.github.io/word_cloud/generated/wordcloud.WordCloud.html

KeyBERT documentation: https://pypi.org/project/keybert/